# Natural language inference

In [ ]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2016 term"

## Contents

0. [Overview](#Overview)
0. [Set-up](#Set-up)
0. [Working with SNLI](#Working-with-SNLI)
   0. [Trees](#Trees)
   0. [Readers](#Readers)
0. [MaxEnt classifier approach](#MaxEnt-classifier-approach)
   0. [Baseline classifier features](#Baseline-classifier-features)
   0. [Classifier training and assessment](#Classifier-training-and-assessment)
   0. [A few ideas for better classifier features](#A-few-ideas-for-better-classifier-features)
0. [Recurrent neural network approach](#Shallow-neural-network-approach)
0. [Some further reading](#Some-further-reading)
0. [Exercises](#Exercises)

## Overview

In the context of NLP/NLU, Natural Language Inference (NLI) is the task of predicting the logical relationships between words, phrases, sentences, (paragraphs, documents, ...). Such relationships are crucial for all kinds of reasoning in natural language: arguing, debating, problem solving, summarization, extrapolation, and so forth. 

NLI is a great task for this course. It requires serious linguistic analysis to do well, there are good publicly available datasets, and there are some natural baselines that help with getting a model up and running, and with understanding the performance of more sophisticated approaches.  NLI was also the topic of [Bill's thesis](http://nlp.stanford.edu/~wcmac/papers/nli-diss.pdf) (he popularized the name "NLI"), so you can forever endear yourself to him by working on it!

We looked at NLI briefly in our word-level entailment bake-off (the `wordentail.ipynb` notebook). The purpose of this codebook is to introduce the problem of NLI more fully in the context of the [Stanford Natural Language Inference](http://nlp.stanford.edu/projects/snli/) corpus (SNLI). We'll explore two general approaches:

* Standard classifiers
* Recurrent neural networks

This should be a good starting point for exploring richer models of NLI.

## Set-up

0. Make sure your environment includes all the requirements for [the cs224u repository](https://github.com/cgpotts/cs224u), especially TensorFlow, which isn't included in the standard Anaconda distribution (but is [easily installed](https://anaconda.org/jjhelmus/tensorflow)).
0. Make sure `snli_sample_src` is pointing to your copy of `semparse_dateparse_data.pickle`, which should be included in the repository in the `nli-data` subfolder.

In [44]:
snli_sample_src = os.path.join('nli-data', 'snli_1.0_cs224u_sample.pickle')

snli_sample = pickle.load(file(snli_sample_src))

In [102]:
import os
import re
import sys
import copy
import pickle
import codecs
import numpy as np
import itertools
from collections import Counter
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_selection import SelectFpr, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import KFold
from sklearn.metrics import precision_recall_fscore_support
import tensorflow as tf
import utils

## Working with SNLI

### Trees

In [69]:
WORD_RE = re.compile(r"([^ \(\)]+)", re.UNICODE)

def str2tree(s):
    """Turns labeled bracketing s into a tree structure (tuple of tuples)"""
    s = WORD_RE.sub(r'"\1",', s)
    s = s.replace("\\", "\\\\")
    s = s.replace(")", "),").strip(",")
    s = s.strip(",")
    return eval(s)

For baseline models, we often want just the words, also called terminal nodes or _leaves_.
This function gives us access to them as a list:

In [47]:
str2tree("( ( A child ) ( is ( playing ( in ( a yard ) ) ) ) )")

(('A', 'child'), ('is', ('playing', ('in', ('a', 'yard')))))

In [63]:
def leaves(t):
    """Returns all of the words (terminal nodes) in tree t"""
    words = []
    for x in t:
        if isinstance(x, basestring):
            words.append(x)
        else:
            words += leaves(x)
    return words

In [49]:
leaves(str2tree("( ( A child ) ( is ( playing ( in ( a yard ) ) ) ) )"))

['A', 'child', 'is', 'playing', 'in', 'a', 'yard']

### Reader

In [50]:
LABELS = ['entailment', 'contradiction', 'neutral']

In [51]:
def snli_reader():
    for d in snli_sample:
        yield (str2tree(d['sentence1_binary_parse']), 
               str2tree(d['sentence2_binary_parse']),
               d['gold_label'])

## MaxEnt classifier approach

### Baseline classifier features

The first baseline we define is the _word overlap_ baseline. It simply uses as
features the words that appear in both sentences.

In [52]:
def word_overlap_phi(t1, t2):
    overlap = [w1 for w1 in leaves(t1) if w1 in leaves(t2)]
    return Counter(overlap)

Another popular baseline is to use as features the full cross-product of
words from both sentences:    

In [53]:
def word_cross_product_phi(t1, t2):
    return Counter([(w1, w2) for w1, w2 in itertools.product(leaves(t1), leaves(t2))])

Both of these feature functions return count dictionaries mapping feature names to 
the number of times they occur in the data. This is the representation we'll work
with throughout; scikit-learn will handle the further processing it needs to build
linear classifiers.

Naturally, you can do better than these feature functions! 
Both of these feature classes might be useful even in a more advanced model, though.

### Building datasets for experiments

The first step in training a classifier is using a feature function like the one above
to turn the data into a list of _training instances_: feature representations and their 
associated labels:

In [85]:
def build_dataset(reader=snli_reader, phi=word_overlap_phi):
    feat_dicts = []
    labels = []
    raw_examples = []
    for t1, t2, label in reader():
        d = phi(t1, t2)
        feat_dicts.append(d)
        labels.append(label)   
        raw_examples.append((t1, t2))
    vectorizer = DictVectorizer(sparse=True)
    feat_matrix = vectorizer.fit_transform(feat_dicts)
    return {'X': feat_matrix, 
            'y': labels, 
            'vectorizer': vectorizer, 
            'raw_examples': raw_examples}

### Training

### Experiments

In [111]:
def experiment(reader=snli_reader, phi=word_overlap_phi, cv=5):
    dataset = build_dataset(reader, phi)
    X = dataset['X']
    y = np.array(dataset['y'])
    mod = LogisticRegression(fit_intercept=True, C=1.0, penalty='l2')
    for train_indices, assess_indices in KFold(len(y), n_folds=cv, shuffle=True):
        X_train, y_train = X[train_indices], y[train_indices]        
        X_assess, y_assess = X[assess_indices], y[assess_indices]
        mod.fit(X_train, y_train)
        predictions = mod.predict(X_assess)
        print precision_recall_fscore_support(y_assess, predictions)             

In [112]:
experiment()

(array([ 0.42920682,  0.51626016,  0.39113924]), array([ 0.5899134 ,  0.43365854,  0.31102164]), array([ 0.49688908,  0.47136797,  0.34650967]), array([1963, 2050, 1987]))
(array([ 0.43262136,  0.48258706,  0.40655941]), array([ 0.55783676,  0.43715573,  0.32751745]), array([ 0.48731409,  0.45874934,  0.36278299]), array([1997, 1997, 2006]))
(array([ 0.43671608,  0.48146067,  0.40780365]), array([ 0.5640648 ,  0.43568887,  0.3246493 ]), array([ 0.49228792,  0.45743261,  0.36150628]), array([2037, 1967, 1996]))
(array([ 0.43084502,  0.49662162,  0.38990536]), array([ 0.56736527,  0.44299347,  0.30822943]), array([ 0.48976955,  0.46827714,  0.34428969]), array([2004, 1991, 2005]))
(array([ 0.42578424,  0.47314715,  0.37795276]), array([ 0.55677839,  0.44160401,  0.28713858]), array([ 0.48254932,  0.45683173,  0.32634561]), array([1999, 1995, 2006]))


### A few ideas for better classifier features

* Cross product of synsets compatible with each word, as given by WordNet. (Here is [a codebook on using WordNet from NLTK to do things like this](http://compprag.christopherpotts.net/wordnet.html).)

* More fine-grained WordNet features &mdash; e.g., spotting pairs like _puppy_/_dog_ across the two sentences.

* Use of other WordNet relations (see Table 1 and Table 2 in [the above codelab](http://compprag.christopherpotts.net/wordnet.html) for relations and their coverage).

* Using the tree structure to define features that are sensitive to how negation scopes over constituents.

* Features that are sensitive to differences in negation between the two sentences.

* Sentiment features seeking to identify contrasting polarity.

## Recurrent neural network approach

## Some further reading

Bowman, Samuel R.; Christopher Potts; and Christopher D. Manning. 2014. 
[Recursive neural networks for learning logical semantics](http://arxiv.org/abs/1406.1827). 
arXiv manuscript 1406.1827. 

Dagan, Ido; Oren Glickman; and  Bernardo Magnini. 2006. 
[The PASCAL recognising textual entailment challenge](http://eprints.pascal-network.org/archive/00001298/01/dagan_et_al_rte05.pdf).
In J. Quinonero-Candela, I. Dagan, B. Magnini, F. d'Alché-Buc, ed., _Machine Learning Challenges_, 
177-190. Springer-Verlag.

Icard, Thomas F. 2012. [Inclusion and exclusion in natural language](http://link.springer.com/article/10.1007%2Fs11225-012-9425-8). _Studia Logica_ 100(4): 705-725.

MacCartney, Bill and Christopher D. Manning. 2009. 
[An extended model of natural logic](http://www.aclweb.org/anthology/W09-3714). 
In  _Proceedings of the Eighth International Conference on Computational Semantics_, 140-156. 
Tilburg, The Netherlands: Association for Computational Linguistics.

## Exercises